In [1]:
code = "ep082020"
hurricane = f"{code}_5day"

In [2]:
import os
import sys
from urllib.request import urlopen, urlretrieve

import lxml.html



def url_lister(url):
    urls = []
    connection = urlopen(url)
    dom = lxml.html.fromstring(connection.read())
    for link in dom.xpath("//a/@href"):
        urls.append(link)
    return urls

def download(url, path):
    sys.stdout.write(fname + "\n")
    if not os.path.isfile(path):
        urlretrieve(
            url,
            filename=path,
            reporthook=progress_hook(sys.stdout)
        )
        sys.stdout.write("\n")
        sys.stdout.flush()


def progress_hook(out):
    """
    Return a progress hook function, suitable for passing to
    urllib.retrieve, that writes to the file object *out*.
    """

    def it(n, bs, ts):
        got = n * bs
        if ts < 0:
            outof = ""
        else:
            # On the last block n*bs can exceed ts, so we clamp it
            # to avoid awkward questions.
            got = min(got, ts)
            outof = "/%d [%d%%]" % (ts, 100 * got // ts)
        out.write("\r  %d%s" % (got, outof))
        out.flush()
    return it

In [3]:
nhc = "http://www.nhc.noaa.gov/gis/forecast/archive/"

fnames = [
    fname for fname in url_lister(nhc)
    if fname.startswith(hurricane) and "latest" not in fname
]

In [4]:
base = os.path.abspath(
    os.path.join(os.path.curdir, "data", hurricane)
)

if not os.path.exists(base):
    os.makedirs(base)

In [5]:
for fname in fnames:
    url = f"{nhc}/{fname}"
    print(url)
    path = os.path.join(base, fname)
    download(url, path)

http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_.zip
ep082020_5day_.zip
  37288/37288 [100%]
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_001.zip
ep082020_5day_001.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_002.zip
ep082020_5day_002.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_003.zip
ep082020_5day_003.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_004.zip
ep082020_5day_004.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_005.zip
ep082020_5day_005.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_006.zip
ep082020_5day_006.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_007.zip
ep082020_5day_007.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_008.zip
ep082020_5day_008.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_009.zip
ep082020_5day_009.zip
http://www.nhc.noaa.gov/gis/forecast/archive//ep082020_5day_010.zip
ep082020_5day_010

In [7]:
import os


os.environ["CPL_ZIP_ENCODING"] = "UTF-8"
os.environ["TZ"] = "GMT0"

In [8]:
from glob import glob
import geopandas


cones, points = [], []
for fname in sorted(
    glob(os.path.join(base, f"{hurricane}_0*.zip"))
):
    number = os.path.splitext(
        os.path.split(fname)[-1])[0].split("_")[-1]
    pgn = geopandas.read_file(
        f"/{code}-{number}_5day_pgn.shp",
        vfs=f"zip://{fname}"
    )
    cones.append(pgn)

    pts = geopandas.read_file(
        f"/{code}-{number}_5day_pts.shp",
        vfs=f"zip://{fname}"
    )
    # Only the first "obsevartion."
    points.append(pts.iloc[0])

/home/filipe/miniconda3/envs/OHW20/lib/python3.8/site-packages/fiona/env.py:398: FionaDeprecationWarning: The vfs keyword argument is deprecated. Instead, pass a URL that uses a zip or tar (for example) scheme.
  return f(*args, **kwargs)


In [9]:
colors = {
    "Subtropical Depression": "yellow",
    "Tropical Depression": "yellow",
    "Tropical Storm": "orange",
    "Subtropical Storm": "orange",
    "Hurricane": "red",
    "Major Hurricane": "crimson"
}

In [10]:
import dateutil


start = points[0]["FLDATELBL"].strip(" HST")
end = points[-1]["FLDATELBL"].strip(" HST")

start = dateutil.parser.parse(start)
end = dateutil.parser.parse(end)

In [11]:
from shapely.geometry import LineString
from shapely.ops import cascaded_union

last_cone = cones[-1]["geometry"].iloc[0]
track = LineString([point["geometry"] for point in points])

polygon = cascaded_union([last_cone, track])

# Add a buffer to find the stations along the track.
bbox = polygon.buffer(2).bounds

In [12]:
strbbox = ", ".join(format(v, ".2f") for v in bbox)
print(f"bbox: {strbbox}\nstart: {start}\n  end: {end}")

bbox: -167.70, 9.80, 196.83, 30.15
start: 2020-07-20 02:00:00
  end: 2020-07-27 23:00:00


In [14]:
import cf_units
from ioos_tools.ioos import collector2table
import pandas as pd
from pyoos.collectors.coops.coops_sos import CoopsSos
from retrying import retry


@retry(stop_max_attempt_number=5, wait_fixed=3000)
def get_coops(start, end, sos_name, units, bbox, verbose=False):
    collector = CoopsSos()
    collector.set_bbox(bbox)
    collector.end_time = end
    collector.start_time = start
    collector.variables = [sos_name]
    ofrs = collector.server.offerings
    title = collector.server.identification.title
    config = dict(
        units=units,
        sos_name=sos_name,
    )

    data = collector2table(
        collector=collector,
        config=config,
        col=f"{sos_name} ({units.format(cf_units.UT_ISO_8859_1)})"
    )

    # Clean the table.
    table = dict(
        station_name=[s._metadata.get("station_name") for s in data],
        station_code=[s._metadata.get("station_code") for s in data],
        sensor=[s._metadata.get("sensor") for s in data],
        lon=[s._metadata.get("lon") for s in data],
        lat=[s._metadata.get("lat") for s in data],
        depth=[s._metadata.get("depth", "NA") for s in data],
    )

    table = pd.DataFrame(table).set_index("station_name")
    if verbose:
        print("Collector offerings")
        print(f"{title}: {len(ofrs)} offerings")
    return data, table

In [17]:
ssh, ssh_table = get_coops(
    start=start,
    end=end,
    sos_name="water_surface_height_above_reference_datum",
    units=cf_units.Unit("meters"),
    bbox=bbox,
)

ConnectTimeout: HTTPSConnectionPool(host='ioos-dif-sos-prod.co-ops-aws-east1.net', port=443): Max retries exceeded with url: /ioos-dif-sos/SOS?service=SOS&version=1.0.0&request=GetObservation&offering=urn%3Aioos%3Anetwork%3ANOAA.NOS.CO-OPS%3AAll&observedProperty=water_surface_height_above_reference_datum&responseFormat=text%2Fcsv&eventTime=2020-07-20T02%3A00%3A00Z%2F2020-07-27T23%3A00%3A00Z&featureOfInterest=BBOX%3A-167.69985133979654%2C9.800675216973772%2C196.8345769420455%2C30.15475 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa7d8fa4e20>, 'Connection to ioos-dif-sos-prod.co-ops-aws-east1.net timed out. (connect timeout=30)'))

In [ ]:
ssh_table.head()

In [ ]:
wind_speed, wind_speed_table = get_coops(
    start=start,
    end=end,
    sos_name="wind_speed",
    units=cf_units.Unit("m/s"),
    bbox=bbox,
)

In [ ]:
wind_speed_table.tail()

In [ ]:
common = set(
    ssh_table["station_code"]
).intersection(wind_speed_table["station_code"])

In [ ]:
ssh_obs, win_obs = [], []
for station in common:
    ssh_obs.extend(
        [
            obs for obs in ssh if
            obs._metadata["station_code"] == station
        ]
    )
    win_obs.extend(
        [
            obs for obs in wind_speed if
            obs._metadata["station_code"] == station
        ]
    )

In [ ]:
index = pd.date_range(
    start=start,
    end=end,
    freq="15min"
)

# Re-index and rename series.
ssh_observations = []
for series in ssh_obs:
    _metadata = series._metadata
    series.index = series.index.tz_localize(None)
    obs = series.reindex(index=index, limit=1, method="nearest")
    obs._metadata = _metadata
    obs.name = _metadata["station_name"]
    ssh_observations.append(obs)

winds_observations = []
for series in win_obs:
    _metadata = series._metadata
    series.index = series.index.tz_localize(None)
    obs = series.reindex(index=index, limit=1, method="nearest")
    obs._metadata = _metadata
    obs.name = _metadata["station_name"]
    winds_observations.append(obs)

In [ ]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.models import Range1d, LinearAxis, HoverTool

from folium import IFrame

# Plot defaults.
tools = "pan,box_zoom,reset"
width, height = 750, 250


def make_plot(ssh, wind):
    p = figure(toolbar_location="above",
               x_axis_type="datetime",
               width=width,
               height=height,
               tools=tools,
               title=ssh.name)

    p.yaxis.axis_label = "wind speed (m/s)"

    l0 = p.line(
        x=wind.index,
        y=wind.values,
        line_width=5,
        line_cap="round",
        line_join="round",
        legend="wind speed (m/s)",
        color="#9900cc",
        alpha=0.5,
    )

    p.extra_y_ranges = {}
    p.extra_y_ranges["y2"] = Range1d(
        start=-1,
        end=3.5
    )

    p.add_layout(
        LinearAxis(
            y_range_name="y2",
            axis_label="ssh (m)"),
        "right"
    )

    l1 = p.line(
        x=ssh.index,
        y=ssh.values,
        line_width=5,
        line_cap="round",
        line_join="round",
        legend="ssh (m)",
        color="#0000ff",
        alpha=0.5,
        y_range_name="y2",
    )

    p.legend.location = "top_left"

    p.add_tools(
        HoverTool(
            tooltips=[
                ("wind speed (m/s)", "@y"),
            ],
            renderers=[l0],
        ),
        HoverTool(
            tooltips=[
                ("ssh (m)", "@y"),
            ],
            renderers=[l1],
        ),
    )
    return p


def make_marker(p, location, fname):
    html = file_html(p, CDN, fname)
    iframe = IFrame(html, width=width+45, height=height+80)

    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color="green", icon="stats")
    marker = folium.Marker(location=location,
                           popup=popup,
                           icon=icon)
    return marker

### Let's assemble all the parts in our map!

In [ ]:
import folium
from folium.plugins import Fullscreen, MarkerCluster
from ioos_tools.ioos import get_coordinates


lon = track.centroid.x
lat = track.centroid.y

m = folium.Map(
    location=[lat, lon], tiles="OpenStreetMap", zoom_start=4)

Fullscreen(
    position="topright", force_separate_button=True).add_to(m)

marker_cluster0 = MarkerCluster(name="Observations")
marker_cluster1 = MarkerCluster(name="Past predictions")
marker_cluster0.add_to(m)
marker_cluster1.add_to(m);

In [ ]:
url = ("http://oos.soest.hawaii.edu/thredds/wms/hioos/"
       "satellite/dhw_5km")
w = folium.WmsTileLayer(
    url,
    name="Sea Surface Temperature",
    fmt="image/png",
    layers="CRW_SST",
    attr="PacIOOS TDS",
    overlay=True,
    transparent=True)

w.add_to(m);

In [ ]:
def style_function(feature):
    return {
        "fillOpacity": 0,
        "color": "black",
        "stroke": 1,
        "weight": 0.5,
        "opacity": 0.2,
    }

# Past cone predictions.
for cone in cones[:-1]:
    folium.GeoJson(
        data=cone.__geo_interface__,
        style_function=style_function,
    ).add_to(marker_cluster1)

# Latest cone prediction.
latest = cones[-1]
folium.GeoJson(
    data=latest.__geo_interface__,
    name=f"Cone prediction as of {latest['ADVDATE'].values[0]}",
).add_to(m);

In [ ]:
# Latest points prediction.
for k, row in pts.iterrows():
    date = row["FLDATELBL"]
    hclass = row["TCDVLP"]
    location = row["LAT"], row["LON"]
    popup = f"{date}<br>{hclass}"
    folium.CircleMarker(
        location=location,
        radius=10,
        fill=True,
        color=colors[hclass],
        popup=popup,
    ).add_to(m);

In [ ]:
# All the points along the track.
for point in points:
    date = point["FLDATELBL"]
    hclass = point["TCDVLP"]
    location = point["LAT"], point["LON"]
    popup = f"{date}<br>{hclass}"
    folium.CircleMarker(
        location=location,
        radius=5,
        fill=True,
        color=colors[hclass],
        popup=popup,
    ).add_to(m);

In [ ]:
# Observations.
for ssh, wind in zip(ssh_observations, winds_observations):
    fname = ssh._metadata["station_code"]
    location = ssh._metadata["lat"], ssh._metadata["lon"]
    p = make_plot(ssh, wind)
    marker = make_marker(p, location=location, fname=fname)
    marker.add_to(marker_cluster0)

folium.LayerControl().add_to(m)

p = folium.PolyLine(get_coordinates(bbox),
                    color="#009933",
                    weight=1,
                    opacity=0.2)

p.add_to(m);

In [ ]:
import warnings
warnings.simplefilter("ignore")


def embed_map(m):
    from IPython.display import HTML
    m.save("index.html")
    with open("index.html") as f:
        html = f.read()

    srcdoc = html.replace('"', '&quot;')
    iframe = f'<iframe srcdoc="{srcdoc}" style="width: 100%; height: 750px; border: none"></iframe>'
    return HTML(iframe)

In [ ]:
embed_map(m)